# Time Transformer [`t2`] (mini) Analysis Pipeline

### Time Transformer Analysis Pipeline

Time Transformer is an attempt to apply the transformer architecture to time series. In this walk through I will use the **WISDM** (**WI**reless **S**ensor **D**ata **M**ining dataset: http://www.cis.fordham.edu/wisdm/includes/files/sensorKDD-2010.pdf)
    

The purpose of this dataset is for multivariate time series classification, which is exactly the problem we are trying to solve for Type Ia Supernova. With Supernova classification we have observations accross 6 passbands for "each timestep". 

There is a ***disclaimer*** is since we don't _actually_ have measurements for all 6 'variables' for everytime step, but this can be resolved with a Gaussian Process regression for interpoloation of points between actual observed data.

Therefore we will use this dataset as a proof on concept to see if this type of architecture would be suitable for our problem too.

Below is a short video showing how data of this type is collected:

In [ ]:
from IPython.display import HTML

HTML('<center><iframe width="560" height="315" src="https://www.youtube.com/embed/XOEN9W05_4A" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe></center>')

### Load in the WISM dataset

See `astronet/t2/utils.py` for details

### Hyperparameter Optimisation

In [ ]:
!python ../astronet/t2/opt/hypertrain.py 

Lets now inspect this hyperparameter optimisation run.

We will first load in this study like so:

In [ ]:
study_name = f"{unixtimestamp}-{label}"
with open(f"{Path(__file__).absolute().parent}/runs/study-{study_name}.pkl", "rb") as sf:
    study = joblib.load(sf)

In [ ]:
# # To load the best performing study out of all studies completed so far, one can run:
# with open(f"{Path(__file__).absolute().parent}/runs/results.json") as f:
#     events = json.load(f)
#     event = max(events['optuna_result'], key=lambda ev: ev['value'])
#     print(event)
    
# # Where study_name would be set via event['name'] like so:
# study_name = event['name']

In [ ]:
from optuna.visualization import *
import matplotlib.pyplot as plt
%matplotlib inline

plot_parallel_coordinate(study)

In [ ]:
plot_optimization_history(study)

In [ ]:
plot_contour(study, params=['embed_dim',
                            'ff_dim', 
                            'num_heads'])

In [ ]:
plot_slice(study)

Now let's train on the best performing study of all hyperparamter optimsation studies. See above for details.

In [ ]:
# %load ../astronet/t2/train.py

In [ ]:
%reset

In [ ]:
# %load ../astronet/t2/evaluate.py

In [ ]:
from astronet.t2.visuals import *

In [ ]:
try:
    log = t2_logger(__file__)
    log.info("_________________________________")
    log.info("File Path:" + str(Path(__file__).absolute()))
    log.info("Parent of Directory Path:" + str(Path().absolute().parent))
except:
    print("Seems you are running from a notebook...")
    __file__ = str(Path().resolve().parent) + "/astronet/t2/visuals.py"

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.serif": ["Computer Modern Roman"]})

mpl.style.use("seaborn")


# Load WISDM-2010
X_train, y_train, X_val, y_val, X_test, y_test = load_WISDM()
# One hot encode y
enc, y_train, y_val, y_test = one_hot_encode(y_train, y_val, y_test)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

with open(str(Path(__file__).absolute().parent) + '/models/results.json') as f:
    events = json.load(f)
    # Get params for best model with highest test accuracy
    event = max(events['training_result'], key=lambda ev: ev['value'])
    print(event)

model_name = event['name']

model = keras.models.load_model(str(Path(__file__).absolute().parent) + f"/models/model-{model_name}")

y_pred = model.predict(X_test)

In [ ]:
plot_history(model_name, event, save=False)

In [ ]:
plot_confusion_matrix(
        model_name,
        enc.inverse_transform(y_test),
        enc.inverse_transform(y_pred),
        enc.categories_[0], 
        save=False
    )

In [ ]:
plot_multiROC(model_name, model, X_test, y_test, enc, save=False)

As mentioned in the [WISDM website](http://www.cis.fordham.edu/wisdm/dataset.php) the dataset can be describes as follows:
    
**Raw Time Series Data**
- Number of examples: 1,098,207
- Number of attributes: 6
- Missing attribute values: None

**Class Distribution**
- Walking: 424,400 (38.6%)
- Jogging: 342,177 (31.2%)
- Upstairs: 122,869 (11.2%)
- Downstairs: 100,427 (9.1%)
- Sitting: 59,939 (5.5%)
- Standing: 48,395 (4.4%)

### Normalize the data

It is important to scale features before training a neural network. Normalization is a common way of doing this scaling. Subtract the mean and divide by the standard deviation of each feature.

In order to work with this data, we will need to do some pre-processing such that all axis are scaled correctly.

Furthermore there is another bit of "pre-processing" required such that we can use `keras` library for Deep Learning and classification. We need to split our dataset into a _train_ and _test_ (and _validation_) set and we will need to **one-hot** encode our targets.

The mean and standard deviation should only be computed using the training data so that the models have no access to the values in the validation and test sets. With time-series data, it is very important not to allow for data to overlap from train to test, i.e we want to avoid the dreaded data leakage problem.

For this we shall first split the data into a train/val/test set, and _then_ apply a windowing function to create small time segments we can then use for training and testing

What we have now is a dataset of the shape above: (22454, 200, 3) (22454, 6) for `X_train` and `y_train` respectivly. 

This is to say we have 22452 _batches_, that are 200 time-step windows, that contain 3 variables each, i.e. `x, y, z` measurements. As you may have guess alreeady, this could be translated to X-number of _batches_, that are X-*MJD* day length windows, each containing 6 pass-band filter measurements.

## What is the Transformer achitecture?

The overall architecture of the Transformer network can be seen below. It consists of the _encoder_ section and a _decoder_ section

<img src="./images/transformer-arch.png" height=40% width=40%>

In the first instance, we will implement just the _encoder_ section show here and add a softmax output:

<img src="./images/transformer-arch-encoder.png" height=30% width=30%>

Depending on results and if we are able to get the inputs to behave, we will move towards development and analysis of the full Transformer architecture for time series.

The above can be seen pictorially as follows:
<img src="./images/transformer-arch-multihead.png" height=30% width=30%>

## Adapting the Transformer for Time-Series

The Transformer architeecture was originally applied to NLP tasks of translation. For this, there were specific stages in the pipeline that are focused towards working with words. For example, the embedding layer would project each word into a high-dimensional vector space followed by a positional enciding layer to ensure sequential information is preversed.

In the case of time-series, a simple Dese layer can be used to map the (`m, T, 6`)(batch, time-steps, features) into a (`m, 32`)(batch, time-steps x features). In `keras`, in order to apply a Dense layer to each input in the time sequence, there is `TimeDistributed` layer.  

From "Hands-On Machine Learning" by Aurelien Geron

> The `TimeDistributed` layer wraps any layer (e.g. `Dense` layer) and applies it at every time step of its input sequence. It does this efficiently, by reshaping the inputs so that each time step is treated as a seperate instance ( i.e, it reshapes the inputs from [*batch size, time-steps, input dimension*] to [*batch size x time-steps, input dimensions].

...
> The `Dense` layer actually supports sequences as inputs (and even higher dimensional inputs): it handles them just like `TimeDistributed(Dense(...))`, meaning it is applied to the last input dimension only (independently accross all time steps). Thus, we could just replace ... with just `Dense(..)`

> Note that a `TimeDistributed(Dense(n))` layer is equivalent to a `Conv1D(n, filter_size=1)` layer.

* https://keras.io/api/layers/convolution_layers/convolution1d/
* https://keras.io/api/layers/recurrent_layers/time_distributed/

##### But what is a 1-D convolition I hear you cry!?

In [ ]:
from IPython.display import HTML

HTML('<center><iframe width="560" height="315" src="https://www.youtube.com/embed/vcp0XvDAX68" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe></center>')

## Comparison to baseline models other approaches

The results above are at least comparable to other experimental results by [Jennifer R. Kwapisz et al 2010](https://www.cis.fordham.edu/wisdm/includes/files/sensorKDD-2010.pdf) where their best reported model of an MLP achieved acc ~ 91%. With notablly better confusion matrix on our part. See tables below:

<img src="images/wisdm-2010-table2.png" height=40% width=40%>

<img src="images/wisdm-2010-table5.png" height=40% width=40%>

Another piece of work that is work comparing against is the analysis done by Jason Brownlee found [here](https://machinelearningmastery.com/how-to-develop-rnn-models-for-human-activity-recognition-time-series-classification/)

The scope of that analysis was to evaluate deep learning methods for a similiar HAR dataset, namely the [Human Activity Recognition Using Smartphones Data Set, UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones)

> The raw data is not available. Instead, a pre-processed version of the dataset was made available. The pre-processing steps included:

> Pre-processing accelerometer and gyroscope using noise filters.
Splitting data into fixed windows of 2.56 seconds (128 data points) with 50% overlap.Splitting of accelerometer data into gravitational (total) and body motion components.
Feature engineering was applied to the window data, and a copy of the data with these engineered features was made available.

> A number of time and frequency features commonly used in the field of human activity recognition were extracted from each window. The result was a 561 element vector of features.

> The dataset was split into train (70%) and test (30%) sets based on data for subjects, e.g. 21 subjects for train and nine for test.

> Experiment results with a support vector machine intended for use on a smartphone (e.g. fixed-point arithmetic) resulted in a predictive accuracy of 89% on the test dataset, achieving similar results as an unmodified SVM implementation.

Jason Brownlee was able to get results on the order of ~ 90% for:
- LSTM --> Accuracy: 89.722% (+/-1.371)
- CNN+LSTM --> Accuracy: 90.689% (+/-1.051)
- ConvLSTM --> Accuracy: 90.801% (+/-0.886)

It is felt that with further training and better hyperparameter optimisation, better can be achieved, but it has been kept in mind that the cope of this analysis is proof of concept.

As such, we will move on to exploring the updated version of **WISDM** released recently.

## WISDM-2019

In late 2019, an [updated **WISDM** dataset](https://archive.ics.uci.edu/ml/datasets/WISDM+Smartphone+and+Smartwatch+Activity+and+Biometrics+Dataset+) was release which included 18 categories with many more observations. It is a big imporovement on the previous dataset, with balanced class distributions shown in table 4 of the [paper](https://archive.ics.uci.edu/ml/machine-learning-databases/00507/WISDM-dataset-description.pdf) and visualised below:

## References

#### Papers:


#### Weblinks:

* [The llustrated Transformer](http://jalammar.github.io/illustrated-transformer/)


#### TODO

   1. Implement a custom callback
   2. ~~Compare LSTM of blog post~~
   3. Put code into t2 files
   4. Explain dimensions, give examples. Windowing etc. use TF time_series.ipynb notebooks as example
       - Build WindowGenerator, time_series.ipynb notebook for examples
   5. Give in-depth explaination to adaption for time-series with relation to Transformer achitecture, use http://nlp.seas.harvard.edu/2018/04/03/attention.html and https://srome.github.io/Understanding-Attention-in-Neural-Networks-Mathematically/ as well as http://jalammar.github.io/illustrated-transformer/ and https://github.com/curiousily/Deep-Learning-For-Hackers/blob/master/13.time-series-human_activity_recognition.ipynb as reference
   6. Add table of contents to this notebook